# Imports

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Widgets


In [0]:
dbutils.widgets.removeAll()

In [0]:
dbutils.widgets.text("storage_name", "adlsmartdatalavs2")
dbutils.widgets.text("container", "raw")
dbutils.widgets.text("catalogo", "chocolate_sales")
dbutils.widgets.text("esquema", "bronze")

# Parameters

In [0]:
storage_name = dbutils.widgets.get("storage_name")
container = dbutils.widgets.get("container")
catalogo = dbutils.widgets.get("catalogo")
esquema = dbutils.widgets.get("esquema")



# CSV Path

In [0]:
route = f"abfss://{container}@{storage_name}.dfs.core.windows.net/sales.csv"

# Schema Structure

In [0]:
sales_schema = StructType([
    StructField("sales_person_id", StringType(), True),
    StructField("country_id", StringType(), True),
    StructField("product_id", StringType(), True),
    StructField("date", DateType(), True),
    StructField("amount", FloatType(), True),
    StructField("boxes_shipped", IntegerType(), True)
])    

# Read Source

In [0]:
df_sales = spark.read.option("header",True).schema(sales_schema).csv(route)

df_sales_final = df_sales.withColumn("load_date", current_timestamp())

# Save in table

In [0]:
df_sales_final.write.mode("overwrite").saveAsTable(f"{catalogo}.{esquema}.sales")